In [ ]:
# https://gymnasium.farama.org/

In [ ]:
import gymnasium as gym

import time
from collections import deque, namedtuple

import numpy as np
import tensorflow as tf
import random

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import MSE
from tensorflow.keras.optimizers import Adam

import base64

import imageio
import IPython
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd

In [ ]:
env = gym.make('LunarLander-v3')
env.reset()

In [ ]:
def compute_loss(experiences, gamma, q_network, target_q_network):
    """ 
    Calculates the loss.
    
    Args:
      experiences: (tuple) tuple of [states, actions, rewards, next_states, done_vals]
      gamma: (float) The discount factor.
      q_network: (tf.keras.Sequential) Keras model for predicting the q_values
      target_q_network: (tf.keras.Sequential) Keras model for predicting the targets
          
    Returns:
      loss: (TensorFlow Tensor(shape=(0,), dtype=int32)) the Mean-Squared Error between
            the y targets and the Q(s,a) values.
    """

    # Unpack the mini-batch of experience tuples
    states, actions, rewards, next_states, done_vals = experiences
    
    # Compute max Q^(s,a)
    max_qsa = tf.reduce_max(target_q_network(next_states), axis=-1)
    
    # y = R if episode terminates, otherwise y = R + γ max Q^(s,a).
    y_targets = rewards + (gamma * max_qsa * (1 - done_vals))
    
    # Get the q_values and reshape to match y_targets
    q_values = q_network(states)
    q_values = tf.gather_nd(
        q_values, 
        tf.stack([
            tf.range(q_values.shape[0]),
            tf.cast(actions, tf.int32)
        ], axis=1))
        
    # Compute the loss with mean squared error
    loss = MSE(y_targets, q_values)
    
    return loss

In [ ]:
def update_target_network(q_network, target_q_network):
    """
    Updates the weights of the target Q-Network using a soft update.
    
    The weights of the target_q_network are updated using the soft update rule:
    
                    w_target = (TAU * w) + (1 - TAU) * w_target
    
    where w_target are the weights of the target_q_network, TAU is the soft update
    parameter, and w are the weights of the q_network.
    
    Args:
        q_network (tf.keras.Sequential): 
            The Q-Network. 
        target_q_network (tf.keras.Sequential):
            The Target Q-Network.
    """

    for target_weights, q_net_weights in zip(
        target_q_network.weights, q_network.weights
    ):
        target_weights.assign(TAU * q_net_weights + (1.0 - TAU) * target_weights)

In [ ]:
@tf.function
def agent_learn(experiences, gamma):
    """
    Updates the weights of the Q networks.
    
    Args:
      experiences: (tuple) tuple of [states, actions, rewards, next_states, done_vals]
      gamma: (float) The discount factor.
    
    """
    
    # Calculate the loss
    with tf.GradientTape() as tape:
        loss = compute_loss(experiences, gamma, q_network, target_q_network)

    # Get the gradients of the loss with respect to the weights.
    gradients = tape.gradient(loss, q_network.trainable_variables)
    
    # Update the weights of the q_network.
    optimizer.apply_gradients(zip(gradients, q_network.trainable_variables))

    # update the weights of target q_network
    update_target_network(q_network, target_q_network)

In [ ]:
def get_action(q_values, epsilon=0.0):
    """
    Returns an action using an ε-greedy policy.

    This function will return an action according to the following rules:
        - With probability epsilon, it will return an action chosen at random.
        - With probability (1 - epsilon), it will return the action that yields the
        maximum Q value in q_values.
    
    Args:
        q_values (tf.Tensor):
            The Q values returned by the Q-Network. For the Lunar Lander environment
            this TensorFlow Tensor should have a shape of [1, 4] and its elements should
            have dtype=tf.float32. 
        epsilon (float):
            The current value of epsilon.

    Returns:
       An action (numpy.int64). For the Lunar Lander environment, actions are
       represented by integers in the closed interval [0,3].
    """

    if random.random() > epsilon:
        return int(tf.argmax(q_values[0], axis=-1).numpy())
    else:
        return env.action_space.sample()

In [ ]:
def check_update_conditions(t, num_steps_to_update, current_buffer_size):
    """
    Determines if the conditions are met to perform a learning update.

    Checks if the current time step t is a multiple of num_steps_to_update and if the
    buffer has enough experience tuples to fill a mini-batch (for example, if the
    mini-batch size is 64, then the buffer should have more than 64 experience
    tuples in order to perform a learning update).
    
    Args:
        t (int):
            The current time step.
        num_steps_to_update (int):
            The number of time steps used to determine how often to perform a learning
            update. A learning update is only performed every num_steps_to_update time steps.
        current_buffer_size (int):
            The number of experiences in the buffer. 

    Returns:
       True if the conditions are met
    """

    return ((t + 1) % num_steps_to_update == 0) and (current_buffer_size > MINIBATCH_SIZE)

In [ ]:
def get_experiences():
    """
    Returns a random vectorized sample of experience tuples drawn from the circular buffer.

    The size of the random sample is determined by the mini-batch size (MINIBATCH_SIZE). 

    Returns:
        A tuple (states, actions, rewards, next_states, done_vals) where:

            - states are the starting states of the agent.
            - actions are the actions taken by the agent from the starting states.
            - rewards are the rewards received by the agent after taking the actions.
            - next_states are the new states of the agent after taking the actions.
            - done_vals are the boolean values indicating if the episode ended.

        All tuple elements are TensorFlow Tensors whose shape is determined by the
        mini-batch size and the given Gym environment. For the Lunar Lander environment
        the states and next_states will have a shape of [MINIBATCH_SIZE, 8] while the
        actions, rewards, and done_vals will have a shape of [MINIBATCH_SIZE]. All
        TensorFlow Tensors have elements with dtype=tf.float32.
    """
    
    assert buffer_size >= MINIBATCH_SIZE, f"buffer size {buffer_size} < batch size {MINIBATCH_SIZE}"
    idx = np.random.randint(0, buffer_size, size=MINIBATCH_SIZE)

    # Vectorized indexing then single tf.convert_to_tensor calls
    states = tf.convert_to_tensor(state_buf[idx], dtype=tf.float32)
    actions = tf.convert_to_tensor(action_buf[idx], dtype=tf.int32)
    rewards = tf.convert_to_tensor(reward_buf[idx], dtype=tf.float32)
    next_states = tf.convert_to_tensor(next_state_buf[idx], dtype=tf.float32)
    done_vals = tf.convert_to_tensor(done_buf[idx].astype(np.uint8), dtype=tf.float32)

    return (states, actions, rewards, next_states, done_vals)

In [ ]:
def get_new_eps(epsilon):
    """
    Updates the epsilon value for the ε-greedy policy.
    
    Gradually decreases the value of epsilon towards a minimum value (E_MIN) using the
    given ε-decay rate (E_DECAY).

    Args:
        epsilon (float):
            The current value of epsilon.

    Returns:
       A float with the updated value of epsilon.
    """

    return max(E_MIN, E_DECAY * epsilon)

In [ ]:
def store_experience(s, a, r, ns, d):
    """Write one experience into a preallocated circular buffer."""
    global buffer_ptr, buffer_size
    state_buf[buffer_ptr] = s
    action_buf[buffer_ptr] = a
    reward_buf[buffer_ptr] = r
    next_state_buf[buffer_ptr] = ns
    done_buf[buffer_ptr] = d
    buffer_ptr = (buffer_ptr + 1) % MEMORY_SIZE
    buffer_size = min(buffer_size + 1, MEMORY_SIZE)

In [ ]:
# Create the Q-Networks
state_size = env.observation_space.shape
num_actions = env.action_space.n

q_network = Sequential([
    Input(shape = state_size),
    Dense(64, activation = 'relu'),
    Dense(64, activation = 'relu'),
    Dense(num_actions, activation = 'linear')
])

target_q_network = Sequential([
    Input(shape = state_size),
    Dense(64, activation = 'relu'),
    Dense(64, activation = 'relu'),
    Dense(num_actions, activation = 'linear')
])

In [ ]:
# Define our hyperparameters
MEMORY_SIZE = 100_000     # size of memory buffer
GAMMA = 0.995             # discount factor
ALPHA = 1e-3              # learning rate  
NUM_STEPS_FOR_UPDATE = 4  # perform a learning update every C time steps
TAU = 1e-3  # Soft update parameter.
MINIBATCH_SIZE = 64
E_DECAY = 0.995  # ε-decay rate for the ε-greedy policy.
E_MIN = 0.01  # Minimum ε value for the ε-greedy policy.

optimizer = Adam(learning_rate=ALPHA)

num_episodes = 2000
max_num_timesteps = 1000

total_point_history = []

num_p_av = 100    # number of total points to use for averaging
epsilon = 1.0     # initial ε value for ε-greedy policy

# Create a circular replay buffer (fast to sample)
state_dim = env.observation_space.shape[0]   # 8 for LunarLander

state_buf = np.zeros((MEMORY_SIZE, state_dim), dtype=np.float32)
action_buf = np.zeros((MEMORY_SIZE,), dtype=np.int32)
reward_buf = np.zeros((MEMORY_SIZE,), dtype=np.float32)
next_state_buf = np.zeros((MEMORY_SIZE, state_dim), dtype=np.float32)
done_buf = np.zeros((MEMORY_SIZE,), dtype=np.uint8)

# Pointers for circular buffer
buffer_ptr = 0      # next write position
buffer_size = 0     # how many valid items currently in buffer

# Set the target network weights equal to the Q-Network weights
target_q_network.set_weights(q_network.get_weights())

In [ ]:
# Train the model
start = time.time()

for i in range(num_episodes):
    
    # Reset the environment to the initial state and get the initial state
    state, info = env.reset()
    assert hasattr(state, '__len__') and (isinstance(state, np.ndarray) or len(state) > 0)
    total_points = 0
    
    for t in range(max_num_timesteps):
        
        # From the current state S choose an action A using an ε-greedy policy
        state_qn = np.expand_dims(state, axis=0)  # state needs to be the right shape for the q_network
        q_values = q_network(state_qn)
        action = get_action(q_values, epsilon)
        
        # Take action A and receive reward R and the next state S'
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = bool(terminated or truncated)
        next_state = np.asarray(next_state, dtype=np.float32)
        
        # Store experience tuple (S,A,R,S') in the memory buffer.
        # We store the done variable as well for convenience.
        store_experience(state, action, reward, next_state, done)
        
        # Only update the network every NUM_STEPS_FOR_UPDATE time steps.
        update = check_update_conditions(t, NUM_STEPS_FOR_UPDATE, buffer_size)
        
        if update:
            # Sample random mini-batch of experience tuples (S,A,R,S') from D
            experiences = get_experiences()
            
            # Set the y targets, perform a gradient descent step,
            # and update the network weights.
            agent_learn(experiences, GAMMA)
        
        state = next_state.copy()
        total_points += reward
        
        if done:
            break
            
    total_point_history.append(total_points)
    av_latest_points = np.mean(total_point_history[-num_p_av:])
    
    # Update the ε value
    epsilon = get_new_eps(epsilon)

    print(f"\rEpisode {i+1} | Total point average of the last {num_p_av} episodes: {av_latest_points:.2f}", end="")

    if (i+1) % num_p_av == 0:
        print(f"\rEpisode {i+1} | Total point average of the last {num_p_av} episodes: {av_latest_points:.2f}")

    # We will consider that the environment is solved if we get an
    # average of 200 points in the last 100 episodes.
    if av_latest_points >= 200.0:
        print(f"\n\nEnvironment solved in {i+1} episodes!")
        q_network.save('lunar_lander_model.h5')
        break
        
tot_time = time.time() - start

print(f"\nTotal Runtime: {tot_time:.2f} s ({(tot_time/60):.2f} min)")

In [ ]:
def plot_history(point_history, **kwargs):
    """
    Plots the total number of points received by the agent after each episode together
    with the moving average (rolling mean). 

    Args:
        point_history (list):
            A list containing the total number of points the agent received after each
            episode.
        **kwargs: optional
            window_size (int):
                Size of the window used to calculate the moving average (rolling mean).
                This integer determines the fixed number of data points used for each
                window. The default window size is set to 10% of the total number of
                data points in point_history, i.e. if point_history has 200 data points
                the default window size will be 20.
            lower_limit (int):
                The lower limit of the x-axis in data coordinates. Default value is 0.
            upper_limit (int):
                The upper limit of the x-axis in data coordinates. Default value is
                len(point_history).
            plot_rolling_mean_only (bool):
                If True, only plots the moving average (rolling mean) without the point
                history. Default value is False.
            plot_data_only (bool):
                If True, only plots the point history without the moving average.
                Default value is False.
    """

    lower_limit = 0
    upper_limit = len(point_history)

    window_size = (upper_limit * 10) // 100

    plot_rolling_mean_only = False
    plot_data_only = False

    if kwargs:
        if "window_size" in kwargs:
            window_size = kwargs["window_size"]

        if "lower_limit" in kwargs:
            lower_limit = kwargs["lower_limit"]

        if "upper_limit" in kwargs:
            upper_limit = kwargs["upper_limit"]

        if "plot_rolling_mean_only" in kwargs:
            plot_rolling_mean_only = kwargs["plot_rolling_mean_only"]

        if "plot_data_only" in kwargs:
            plot_data_only = kwargs["plot_data_only"]

    points = point_history[lower_limit:upper_limit]

    # Generate x-axis for plotting.
    episode_num = [x for x in range(lower_limit, upper_limit)]

    # Use Pandas to calculate the rolling mean (moving average).
    rolling_mean = pd.DataFrame(points).rolling(window_size).mean()

    plt.figure(figsize=(10, 7), facecolor="white")

    if plot_data_only:
        plt.plot(episode_num, points, linewidth=1, color="cyan")
    elif plot_rolling_mean_only:
        plt.plot(episode_num, rolling_mean, linewidth=2, color="magenta")
    else:
        plt.plot(episode_num, points, linewidth=1, color="cyan")
        plt.plot(episode_num, rolling_mean, linewidth=2, color="magenta")

    text_color = "black"

    ax = plt.gca()
    ax.set_facecolor("black")
    plt.grid()
    plt.xlabel("Episode", color=text_color, fontsize=30)
    plt.ylabel("Total Points", color=text_color, fontsize=30)
    yNumFmt = mticker.StrMethodFormatter("{x:,}")
    ax.yaxis.set_major_formatter(yNumFmt)
    ax.tick_params(axis="x", colors=text_color)
    ax.tick_params(axis="y", colors=text_color)
    plt.show()

In [ ]:
# Plot the total point history along with the moving average
plot_history(total_point_history)

In [ ]:
# Show the model in action
# Suppress warnings from imageio
import logging
logging.getLogger().setLevel(logging.ERROR)
def embed_mp4(filename):
    """
    Embeds an MP4 video file in a Jupyter notebook.
    
    Args:
        filename (string):
            The path to the the MP4 video file that will be embedded (i.e.
            "./videos/lunar_lander.mp4").
    
    Returns:
        Returns a display object from the given video file. This will result in the
        video being displayed in the Jupyter Notebook.
    """

    video = open(filename, "rb").read()
    b64 = base64.b64encode(video)
    tag = """
    <video width="840" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>""".format(
        b64.decode()
    )

    return IPython.display.HTML(tag)

In [ ]:
def create_video(filename, env, q_network, fps=30):
    """
    Creates a video of an agent interacting with a Gym environment.

    The agent will interact with the given env environment using the q_network to map
    states to Q values and using a greedy policy to choose its actions (i.e it will
    choose the actions that yield the maximum Q values).
    
    The video will be saved to a file with the given filename. The video format must be
    specified in the filename by providing a file extension (.mp4, .gif, etc..). If you 
    want to embed the video in a Jupyter notebook using the embed_mp4 function, then the
    video must be saved as an MP4 file. 
    
    Args:
        filename (string):
            The path to the file to which the video will be saved. The video format will
            be selected based on the filename. Therefore, the video format must be
            specified in the filename by providing a file extension (i.e.
            "./videos/lunar_lander.mp4"). To see a list of supported formats see the
            imageio documentation: https://imageio.readthedocs.io/en/v2.8.0/formats.html
        env (Gym Environment): 
            The Gym environment the agent will interact with.
        q_network (tf.keras.Sequential):
            A TensorFlow Keras Sequential model that maps states to Q values.
        fps (int):
            The number of frames per second. Specifies the frame rate of the output
            video. The default frame rate is 30 frames per second.  
    """

    frames = []
    obs, _ = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        frames.append(env.render())
        done = terminated or truncated
    
    # write mp4 using ffmpeg backend
    imageio.mimwrite(filename, frames, fps=30, macro_block_size=None)  # macro_block_size=None avoids size warnings

In [ ]:
env = gym.make('LunarLander-v3', render_mode='rgb_array')
filename = "./lunar_lander.mp4"
create_video(filename, env, q_network)
embed_mp4(filename)